In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [12]:
df = pd.read_csv('./df.csv')

In [36]:
# Spatial join district metadata with original df

# Load the county shapefile
county_shapefile_path = "./Data/VILLAGE_NLSC_121_1130807.shp"
counties = gpd.read_file(county_shapefile_path) 

# Convert to the same CRS as flood data
counties = counties.to_crs("EPSG:3826")  # Ensure CRS is correct

# Aggregate to the town level before using it further
towns_gdf = counties.dissolve(by=["COUNTYNAME", "TOWNNAME"], as_index=False)

# Convert flood data into a GeoDataFrame
df['geometry'] = gpd.points_from_xy(df['Longitude'], df['Latitude'])  # Fast version
flood_gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:3824")

# Perform spatial join to get county names
flood_with_county = gpd.sjoin(flood_gdf, towns_gdf, how="left", predicate="within")

# Extract district name and append back to original df
df['county'] = flood_with_county['COUNTYNAME']
df['town'] = flood_with_county['TOWNNAME']
df['vil'] = flood_with_county['VILLNAME']
df['district'] = df['county'].astype(str).str.cat(df['town'].astype(str))

# Ensure 'district' column in towns_gdf
towns_gdf['district'] = towns_gdf['COUNTYNAME'].astype(str).str.cat(towns_gdf['TOWNNAME'].astype(str))

# Merge town-level geometries into df
df = df.drop(columns=['geometry'])
df = df.merge(towns_gdf[['district', 'geometry']], on='district', how='left')

DataSourceError: ./Data/VILLAGE_NLSC_121_1130807.shp: No such file or directory

In [15]:
# Merge district-level area data

area = pd.read_excel('./Data/area.xlsx')
df = df.merge(area, on='district', how='left')

In [17]:
# Create scaling factor using the number of villages in each district

village_counts = counties.groupby(["COUNTYNAME", "TOWNNAME"], as_index=False)["VILLNAME"].nunique()
village_counts.rename(columns={"VILLNAME": "factor"}, inplace=True)
village_counts["district"] = village_counts["COUNTYNAME"] + village_counts["TOWNNAME"]

df = df.merge(village_counts[["district", "factor"]], on="district", how="left")

In [22]:
df.head()

,station_id,timestamp,value,Longitude,Latitude,SIUnit,county,town,vil,district,geometry,area,factor
0,d83ac636-3d28-43fe-96a9-5c33dde8aebe,2022-07-21 00:08:57.2,0.001,120.691,23.9032,cm,南投縣,南投市,軍功里,南投縣南投市,"POLYGON ((120.70056 23.887, 120.70054 23.88697...",71602100,34
1,d83ac636-3d28-43fe-96a9-5c33dde8aebe,2022-07-21 00:18:57.382,0.001,120.691,23.9032,cm,南投縣,南投市,軍功里,南投縣南投市,"POLYGON ((120.70056 23.887, 120.70054 23.88697...",71602100,34
2,d83ac636-3d28-43fe-96a9-5c33dde8aebe,2022-07-21 00:28:58.358,0.001,120.691,23.9032,cm,南投縣,南投市,軍功里,南投縣南投市,"POLYGON ((120.70056 23.887, 120.70054 23.88697...",71602100,34
3,d83ac636-3d28-43fe-96a9-5c33dde8aebe,2022-07-21 00:38:58.793,0.001,120.691,23.9032,cm,南投縣,南投市,軍功里,南投縣南投市,"POLYGON ((120.70056 23.887, 120.70054 23.88697...",71602100,34
4,d83ac636-3d28-43fe-96a9-5c33dde8aebe,2022-07-21 00:48:59.713,0.001,120.691,23.9032,cm,南投縣,南投市,軍功里,南投縣南投市,"POLYGON ((120.70056 23.887, 120.70054 23.88697...",71602100,34


In [30]:
# Identify flood events

# Time gap threshold and depth threshold
TIME_GAP_THRESHOLD = pd.Timedelta(hours=24)
DEPTH_THRESHOLD = 50
DEPTH_OUTLIER = 300

# Ensure timestamp format
df['timestamp'] = df['timestamp'].str.split('.').str[0] 
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')

# Sort data by location and timestamp
df = df.sort_values(by=['district', 'timestamp'])

# Identify Flood Incidents (Group by district and timestamp while considering station_id)
df['time_diff'] = df.groupby(['district'])['timestamp'].diff()
df['new_incident'] = (df['time_diff'] > TIME_GAP_THRESHOLD).fillna(False)

# Assign an incident group at the district level, ignoring different station IDs
df['incident_group'] = df.groupby(['district'])['new_incident'].cumsum()

# Assign a unique ID based on district and start time of the incident
incident_keys = df.groupby(['district', 'incident_group'])['timestamp'].transform('min').astype(str) + '_' + df['district']
df['incident_id'] = pd.factorize(incident_keys)[0] + 1  # Assign unique numeric ID

# Aggregate incidents, merging across stations in the same district
flood_incidents = df.groupby(['district', 'incident_id']).agg(
    start_time=('timestamp', 'min'),
    end_time=('timestamp', 'max'),
    min_flood_depth=('value', 'min'),
    max_flood_depth=('value', 'max'),
    avg_flood_depth=('value', 'mean'),
    area=('area', 'first'),
    county=('county', 'first'),
    town=('town', 'first'),
    vil=('vil', 'first'),
    geometry=('geometry', 'first'),
    factor=('factor', 'first')
).reset_index()

# Filter incidents with outlier flood depth and exceeding the threshold
flood_incidents = flood_incidents[
    (flood_incidents['max_flood_depth'] < DEPTH_OUTLIER) &
    (flood_incidents['avg_flood_depth'] > DEPTH_THRESHOLD)
]

In [31]:
flood_incidents

,district,incident_id,start_time,end_time,min_flood_depth,max_flood_depth,avg_flood_depth,area,county,town,vil,geometry,factor
264,嘉義市東區,265,2022-10-01 09:00:00,2022-10-01 10:27:48,294.2,295.1,294.900000,30155600,嘉義市,東區,仁義里,"POLYGON ((120.45899 23.4542, 120.45889 23.4541...",39
383,嘉義縣六腳鄉,384,2020-03-21 20:09:29,2020-03-21 20:09:29,77.3,77.3,77.300000,62261900,嘉義縣,六腳鄉,古林村,"POLYGON ((120.28176 23.49113, 120.28066 23.491...",25
385,嘉義縣六腳鄉,386,2020-04-13 21:26:57,2020-04-13 21:26:57,75.7,75.7,75.700000,62261900,嘉義縣,六腳鄉,古林村,"POLYGON ((120.28176 23.49113, 120.28066 23.491...",25
388,嘉義縣六腳鄉,389,2020-05-06 22:38:20,2020-05-06 22:38:20,78.0,78.0,78.000000,62261900,嘉義縣,六腳鄉,古林村,"POLYGON ((120.28176 23.49113, 120.28066 23.491...",25
448,嘉義縣大林鎮,449,2021-07-15 16:13:26,2021-07-15 17:52:26,38.6,142.1,101.158824,64166300,嘉義縣,大林鎮,上林里,"POLYGON ((120.46465 23.58248, 120.46463 23.582...",21
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7030,高雄市美濃區,7031,2020-09-09 02:55:02,2020-09-09 13:21:51,143.0,156.0,147.344828,120031600,高雄市,美濃區,合和里,"POLYGON ((120.54604 22.82919, 120.54544 22.829...",19
7089,高雄市路竹區,7090,2021-03-12 14:25:02,2021-03-12 17:56:04,9.0,199.0,94.222222,48434800,高雄市,路竹區,文北里,"POLYGON ((120.25677 22.82859, 120.25648 22.829...",20
7090,高雄市路竹區,7091,2021-05-05 16:06:31,2021-05-05 16:18:04,198.0,200.0,199.000000,48434800,高雄市,路竹區,文北里,"POLYGON ((120.25677 22.82859, 120.25648 22.829...",20
7101,高雄市鳥松區,7102,2021-01-28 00:06:05,2021-01-28 10:46:05,68.0,96.0,81.953846,24592700,高雄市,鳥松區,鳥松里,"POLYGON ((120.39991 22.64375, 120.39957 22.643...",7


In [32]:
# Export as csv
flood_incidents.to_csv("flood_incidents.csv", index=False)